# BUILDING COMPLEX MODELS WITH THE Functional API

* Example: **_Wide & Deep_** neural network ([Heng-Tze Cheng et al. 2016](https://dl.acm.org/doi/10.1145/2988450.2988454))
  * Connects all or parts of the inputs **directly to the output layer** (see fig. 10-14 in book)
  * Architecture that makes possible for the nn to learn both:
    * **deep patterns** (using the deep path)
    * **simple rules** (through the short path) -> usually distorted by sequence of transformations

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

# Loading the dataset

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, X_train_full.dtype)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
print(class_names[y_train[0]])



(60000, 28, 28) uint8
Coat


# Building the model

### Single input

<img src="images/ch-12-fig-10-14-functional-wide-deep-single-input.jpg" width="350">

In [38]:
# specifying shape and dtype of input layer
input_ = keras.layers.Input(shape=X_train.shape[1:])  # (28, 28)

# create first hidden layer and call it like a function, passing it the input_
#   = 'Functional' API: We are just telling Keras how to connect the layers, no data processed yet
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)

# concatenate input with output of second hidden layer
concat = keras.layers.Concatenate()([input_, hidden2])  # concatenate tensors of same shape

# single neuron, no activation function
output = keras.layers.Dense(1)(concat)

model = keras.Model(inputs=[input_], outputs=[output])

### Multiple inputs

<img src="images/ch-12-fig-10-15-functional-wide-deep-multiple-inputs.jpg" width="350">

In [45]:
input_A = keras.layers.Input(shape=[5], name='wide_input') # Send 5 feats (0 to 4) through wide path
input_B = keras.layers.Input(shape=[6], name='deep_input') # Send 6 feats (2 to 7) through deep path

hidden1 = keras.layers.Dense(30, activation='relu', name='hidden1')(input_B)
hidden2 = keras.layers.Dense(30, activation='relu', name='hidden2')(hidden1)
print(input_B.shape, hidden1.shape)

concat = keras.layers.concatenate([input_A, hidden2])

output = keras.layers.Dense(1, name='output')(concat)

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

(None, 6) (None, 30)


# Compiling, training, evaluating, testing and predicting

In [46]:
model.compile(loss='mse',
              optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]    # input_A and input_B
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A,  X_test_B  = X_test[:, :5],  X_test[:, 2:]
X_new_A,   X_new_B   = X_test_A[:3],   X_test_B[:3]

model.summary()

history = model.fit( (X_train_A, X_train_B),
                     y_train,
                     epochs=20,
                     validation_data=((X_valid_A, X_valid_B), y_valid) )

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

(55000, 5, 28) (55000, 26, 28)
Model: "functional_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
hidden1 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
hidden2 (Dense)                 (None, 30)           930         hidden1[0][0]                    
_______________________________________________________

ValueError: in user code:

    /Users/macbook/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/macbook/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/macbook/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/macbook/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/macbook/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/macbook/anaconda3/lib/python3.7/site-packages/t

### Multiple outputs